# Отчёт по внедрению гидрологической коррекции ЦМП в геопортал

## Решённые задачи

1. **Интеграция алгоритма гидрологической коррекции DEM (ЦМП) в Django-бэкенд**
   - Реализована функция для автоматической гидрологической коррекции DEM с помощью библиотеки pysheds.

2. **Создание REST API для запуска коррекции DEM**
   - Добавлен endpoint `/api/hydro-correction/` для запуска коррекции DEM через POST-запрос.

3. **Графический интерфейс для загрузки и обработки DEM**
   - Добавлена модель DEMFile и интеграция с Django Admin для загрузки DEM-файлов и запуска коррекции через админку.

4. **Миграции для новой модели DEMFile**
   - Создана миграция для хранения информации о DEM-файлах и результатах коррекции.


## Код для каждой задачи

### 1. Функция гидрологической коррекции DEM

```python

def hydrological_dem_correction(dem_path, output_dem_path=None, output_acc_path=None):
    """
    Гидрологическая коррекция ЦМП (DEM) с помощью pysheds.
    Args:
        dem_path: путь к исходному DEM (GeoTIFF)
        output_dem_path: путь для сохранения скорректированного DEM (опционально)
        output_acc_path: путь для сохранения карты аккумуляции (опционально)
    Returns:
        dict с numpy-массивами скорректированного DEM и аккумуляции
    """
    try:
        from pysheds.grid import Grid
        import numpy as np
        import os
        
        grid = Grid.from_raster(dem_path)
        dem = grid.read_raster(dem_path)

        pit_filled_dem = grid.fill_pits(dem)
        flooded_dem = grid.fill_depressions(pit_filled_dem)
        inflated_dem = grid.resolve_flats(flooded_dem)
        fdir = grid.flowdir(inflated_dem)
        acc = grid.accumulation(fdir)

        # скорректированный DEM
        if output_dem_path:
            grid.save_raster(output_dem_path, inflated_dem, dtype='float32')
        # Сохраняем аккумуляцию
        if output_acc_path:
            grid.save_raster(output_acc_path, acc, dtype='float32')

        return {
            'corrected_dem': inflated_dem,
            'accumulation': acc
        }
    except Exception as e:
        logger.error(f"Ошибка гидрологической коррекции DEM: {str(e)}")
        raise
```

### 2. REST API для запуска коррекции DEM

```python
class HydrologicalCorrectionAPIView(APIView):
    permission_classes = [permissions.IsAdminUser]

    def post(self, request):
        """
        Принимает путь к DEM-файлу (или файл), выполняет гидрологическую коррекцию и возвращает путь к результату.
        Пример запроса: { "dem_path": "/path/to/dem.tif" }
        """
        dem_path = request.data.get('dem_path')
        if not dem_path or not os.path.exists(dem_path):
            return Response({"error": "DEM файл не найден"}, status=status.HTTP_400_BAD_REQUEST)

        output_dir = os.path.join(settings.MEDIA_ROOT, 'dem_results')
        os.makedirs(output_dir, exist_ok=True)
        base_name = os.path.splitext(os.path.basename(dem_path))[0]
        corrected_path = os.path.join(output_dir, f"{base_name}_corrected.tif")
        acc_path = os.path.join(output_dir, f"{base_name}_accumulation.tif")

        try:
            result = hydrological_dem_correction(dem_path, corrected_path, acc_path)
            return Response({
                "corrected_dem": corrected_path.replace(settings.MEDIA_ROOT, settings.MEDIA_URL),
                "accumulation": acc_path.replace(settings.MEDIA_ROOT, settings.MEDIA_URL),
                "min_elevation": float(result['corrected_dem'].min()),
                "max_elevation": float(result['corrected_dem'].max()),
                "max_accumulation": float(result['accumulation'].max())
            })
        except Exception as e:
            return Response({"error": str(e)}, status=status.HTTP_500_INTERNAL_SERVER_ERROR)
```

### 3. Модель DEMFile и интеграция с Django Admin

```python
class DEMFile(models.Model):
    file = models.FileField(upload_to='dem/')
    uploaded_at = models.DateTimeField(auto_now_add=True)
    processed = models.BooleanField(default=False)
    corrected_file = models.FileField(upload_to='dem_results/', null=True, blank=True)
    accumulation_file = models.FileField(upload_to='dem_results/', null=True, blank=True)

    def __str__(self):
        return f"DEM: {self.file.name} (processed: {self.processed})"
```

```python
@admin.register(DEMFile)
class DEMFileAdmin(admin.ModelAdmin):
    list_display = ('file', 'uploaded_at', 'processed')
    actions = ['run_hydro_correction']

    def run_hydro_correction(self, request, queryset):
        for dem in queryset:
            if not dem.processed:
                base_name = os.path.splitext(os.path.basename(dem.file.name))[0]
                corrected_path = f"dem_results/{base_name}_corrected.tif"
                acc_path = f"dem_results/{base_name}_accumulation.tif"
                abs_corrected = os.path.join(settings.MEDIA_ROOT, corrected_path)
                abs_acc = os.path.join(settings.MEDIA_ROOT, acc_path)
                os.makedirs(os.path.dirname(abs_corrected), exist_ok=True)
                hydrological_dem_correction(dem.file.path, abs_corrected, abs_acc)
                dem.corrected_file.name = corrected_path
                dem.accumulation_file.name = acc_path
                dem.processed = True
                dem.save()
    run_hydro_correction.short_description = "Выполнить гидрологическую коррекцию DEM"
```

## Диаграмма классов

**DEMFile** — новая сущность, связанная с хранением и обработкой DEM-растров:

- `file: FileField` — исходный DEM-файл
- `uploaded_at: DateTimeField` — дата загрузки
- `processed: BooleanField` — признак обработки
- `corrected_file: FileField` — скорректированный DEM
- `accumulation_file: FileField` — карта аккумуляции

**FloodZone, FloodEvent, MeasurementPoint, WaterLevelMeasurement** — существующие модели для хранения зон затопления, событий, точек измерения и самих измерений.

**Связи:**
- DEMFile не связан напрямую с другими моделями, но его результаты могут использоваться для анализа зон затопления и событий.

### Описание диаграммы классов

- **DEMFile** — самостоятельная сущность для загрузки и хранения DEM и результатов коррекции.
- **FloodZone** — зона затопления (Polygon), может быть получена на основе анализа DEM.
- **FloodEvent** — событие затопления (MultiPolygon), может быть связано с анализом DEM и спутниковых снимков.
- **MeasurementPoint** — точка измерения уровня воды (Point).
- **WaterLevelMeasurement** — измерение уровня воды, связано с MeasurementPoint.